We first get necessary external data and code

In [4]:
!git clone https://github.com/AllenInstitute/deepinterpolation.git
!mkdir -p ephys

fatal: destination path 'deepinterpolation' already exists and is not an empty directory.


Install deepinterpolation package

In [19]:
!pip install git+https://github.com/AllenInstitute/deepinterpolation.git

  Cloning https://github.com/AllenInstitute/deepinterpolation.git to /private/var/folders/hy/9t9mb1ln50z59qlgy6nc7y7r3s796j/T/pip-req-build-zthhkwzm
  Created wheel for deepinterpolation: filename=deepinterpolation-0.1.0-py3-none-any.whl size=21725 sha256=10259817285bfa10cf2d9ab267b92bca6aed56e616fb5a36de6f91afc5e7ac40
  Stored in directory: /private/var/folders/hy/9t9mb1ln50z59qlgy6nc7y7r3s796j/T/pip-ephem-wheel-cache-ullpkl4l/wheels/8e/c0/d8/0c84568edf2d461bf07b4e88eacb17b4ca3765e331f319c3b6
Successfully built deepinterpolation


In [1]:
import deepinterpolation as de
import sys
from shutil import copyfile
import os
from deepinterpolation.generic import JsonSaver, ClassLoader
import datetime
from typing import Any, Dict
import pathlib
import sys

This is used for record-keeping


In [2]:
now = datetime.datetime.now()
run_uid = now.strftime("%Y_%m_%d_%H_%M")

Initialize meta-parameters objects

In [3]:
training_param = {}
generator_param = {}
network_param = {}
generator_test_param = {}

An epoch is defined as the number of batches pulled from the dataset. Because our datasets are VERY large. Often, we cannot
go through the entirity of the data so we define an epoch slightly differently than is usual.

In [4]:
steps_per_epoch = 10

Those are parameters used for the Validation test generator. Here the test is done on the beginning of the data but
this can be a separate file

In [5]:
generator_test_param["type"] = "generator"  # type of collection
generator_test_param["name"] = "EphysGenerator"  # Name of object in the collection
generator_test_param[
    "pre_post_frame"
] = 30  # Number of frame provided before and after the predicted frame
generator_test_param["train_path"] = os.path.join(
    "deepinterpolation",
    "sample_data",
    "ephys_tiny_continuous.dat2",
)
generator_test_param["batch_size"] = 100
generator_test_param["start_frame"] = 0
generator_test_param["end_frame"] = 1999
generator_test_param[
    "pre_post_omission"
] = 1  # Number of frame omitted before and after the predicted frame
generator_test_param["steps_per_epoch"] = steps_per_epoch

Those are parameters used for the main data generator

In [6]:
generator_param["type"] = "generator"
generator_param["steps_per_epoch"] = steps_per_epoch
generator_param["name"] = "EphysGenerator"
generator_param["pre_post_frame"] = 30
generator_param["train_path"] = os.path.join(
    "deepinterpolation",
    "sample_data",
    "ephys_tiny_continuous.dat2",
)
generator_param["batch_size"] = 100
generator_param["start_frame"] = 2000
generator_param["end_frame"] = 7099
generator_param["pre_post_omission"] = 1

Those are parameters used for the training process

In [7]:
training_param["type"] = "trainer"
training_param["name"] = "transfer_trainer"
training_param["run_uid"] = run_uid

# Path to model to transfer and fine-tune
training_param["model_path"] = "ephys/unet_single_ephys_1024_mean_absolute_error_2020_11_12_18_34_2020_11_12_18_34/2020_11_12_18_34_unet_single_ephys_1024_mean_absolute_error_2020_11_12_18_34_model.h5"
training_param["batch_size"] = generator_test_param["batch_size"]
training_param["steps_per_epoch"] = steps_per_epoch
training_param[
    "period_save"
] = 25  # network model is potentially saved during training between a regular nb epochs
training_param["nb_gpus"] = 0
training_param["apply_learning_decay"] = 0
training_param[
    "nb_times_through_data"
] = 1  # if you want to cycle through the entire data. Two many iterations will cause noise overfitting
training_param["learning_rate"] = 0.0001
training_param["pre_post_frame"] = generator_test_param["pre_post_frame"]
training_param["loss"] = "mean_absolute_error"
training_param[
    "nb_workers"
] = 1  # this is to enable multiple threads for data generator loading. Useful when this is slower than training

training_param["model_string"] = (
    "transfer"
    + "_"
    + training_param["loss"]
    + "_"
    + training_param["run_uid"]
)

Where do you store ongoing training progress

In [8]:
 jobdir = os.path.join(
    "ephys", training_param["model_string"] + "_" + run_uid,
)
training_param["output_dir"] = jobdir

try:
    os.mkdir(jobdir)
except:
    print("folder already exists")

Here we create all json files that are fed to the training. This is used for recording purposes as well as input to the training proces

In [9]:
path_training = os.path.join(jobdir, "training.json")
json_obj = JsonSaver(training_param)
json_obj.save_json(path_training)

path_generator = os.path.join(jobdir, "generator.json")
json_obj = JsonSaver(generator_param)
json_obj.save_json(path_generator)

path_test_generator = os.path.join(jobdir, "test_generator.json")
json_obj = JsonSaver(generator_test_param)
json_obj.save_json(path_test_generator)

path_network = os.path.join(jobdir, "network.json")
json_obj = JsonSaver(network_param)
json_obj.save_json(path_network)

Here we create all objects for training.

In [11]:
# We find the generator obj in the collection using the json file
generator_obj = ClassLoader(path_generator)
generator_test_obj = ClassLoader(path_test_generator)

# We find the training obj in the collection using the json file
trainer_obj = ClassLoader(path_training)

# We build the generators object. This will, among other things, calculate normalizing parameters.
train_generator = generator_obj.find_and_build()(path_generator)
test_generator = generator_test_obj.find_and_build()(path_test_generator)

# We build the training object.
training_class = trainer_obj.find_and_build()(train_generator, test_generator, path_training)

Start training. This can take very long time.

In [12]:
training_class.run()

Epoch 1/5
10/10 [==============================] - 22s 2s/step - loss: 0.3887 - val_loss: 0.3639
Epoch 2/5
10/10 [==============================] - 23s 2s/step - loss: 0.3559 - val_loss: 0.3627
Epoch 3/5
10/10 [==============================] - 26s 3s/step - loss: 0.3497 - val_loss: 0.3607
Epoch 4/5
10/10 [==============================] - 23s 2s/step - loss: 0.3678 - val_loss: 0.3593
Epoch 5/5
10/10 [==============================] - 23s 2s/step - loss: 0.3724 - val_loss: 0.3565


Finalize and save output of the training.

In [13]:
training_class.finalize()

Saved model to disk
